# Load libraries and Themes

Run in user_R

In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(Seurat)
        library(ggplot2)
        library(tidyverse)
        library(harmony)
        library(SeuratDisk)
        library(DESeq2)
        library(dplyr)
        library(Rsamtools)
        library(svglite)
        library(viridis)
        library(ComplexHeatmap)
        library(circlize)
        library(CellChat)
        library(patchwork)
        library(reticulate)
        library(svglite)
        library(cowplot)
        library(gridExtra)
        library(pals)
        library(RColorBrewer)
    })})

In [ ]:
#Color Palettes

palette.doublets <- c(
    "#197278",#turqu
    "#C1C1C1"#grey
)

palette.2 <- c(
    "#990902",
    "#0303a3"
)

palette.4 <- DiscretePalette(4, palette = "stepped", shuffle = TRUE)

palette.12 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette.15 <- DiscretePalette(12, palette = "stepped", shuffle = TRUE)

palette <- c(
    "#990902",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#660202",#PER
    "#4b048a",#MESO
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902"#BC
)

palette_CMFBTC <- c(
    "#990902",#CM_0
    "#ebe6c7",#FB
    "#0FB602"#TC
)

palette_split <- c(
    "#C1C1C1",#CM_0
    "#0303a3",#EC-end
    "#ebe6c7",#FB
    "#077a01",#MΦ-M2
    "#05b1eb",#EC-cap
    "#990902",#PER
    "#4b048a",#MESO
    "#7d0701",#CM_1
    "#0FB602",#TC
    "#0324fc",#EC-lym
    "#FFD100",#SC
    "#d90f04",#CM_2
    "#aad902",#BC
    "#660202"#CM_3

)
   
palette.treatment <- c(
    "#AF0000",#ALDO
    "#C1C1C1"#CTRL
)

In [ ]:
umap_theme <- theme(
  axis.line=element_blank(),
  axis.text.x=element_blank(),
  axis.text.y=element_blank(),
  axis.ticks=element_blank(),
  axis.title.x=element_blank(),
  axis.title.y=element_blank(),
  panel.background=element_blank(),
  panel.border=element_blank(),
  panel.grid.major=element_blank(),
  panel.grid.minor=element_blank()
)

vln_theme_1 <- theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

vln_theme_2 <- theme(
  panel.background = element_blank(),
  axis.title.x = element_blank(),
  axis.title.y = element_blank(),
  panel.border = element_blank(),
  panel.grid.major = element_blank(),
  panel.grid.minor = element_blank(),
  text = element_text(size = 25)
)

In [ ]:
setwd("/media/daten/dmeral/scseq_analysis/2024_LA_CTRL_ALDO")

In [ ]:
set.seed(1234)

# CellChat with Subclusters

## Pre-processing

In [ ]:
obj_rename <- LoadH5Seurat("seurat_objects/obj_rename_withsubcluster.h5seurat")

In [ ]:
Idents(obj_rename) <- obj_rename@meta.data$"cell_type"

In [ ]:
LA  <- obj_rename

In [ ]:
print("### Cellnumbers per cluster and sample")
table <- table(LA@meta.data$treatment,
      LA@meta.data$cell_type)
table

In [ ]:
split_LA <- SplitObject(LA, split.by = "treatment")

## CellChat

In [ ]:
# Load after CellChat finished for all 2 conditions
cc_LA_CTRL <- readRDS("CellChat/CellChat_LA_CTRL")
cc_LA_ALDO <- readRDS("CellChat/CellChat_LA_ALDO")

In [ ]:
object.list <- list(LA_CTRL = cc_LA_CTRL, LA_ALDO = cc_LA_ALDO)

cellchat <- mergeCellChat(object.list, add.names = names(object.list))
# Merge the following slots: 'data.signaling','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.
cellchat

object.list_LA <- list(LA_CTRL = cc_LA_CTRL, LA_ALDO = cc_LA_ALDO)
cellchat_LA <- mergeCellChat(object.list_LA, add.names = names(object.list_LA))

## CellChat LA ALDO

In [ ]:
data.input <- GetAssayData(split_LA$ALDO, assay = "RNA", slot = "data")
labels <- Idents(split_LA$ALDO)
meta <- data.frame(group = labels, row.names = names(labels))
cc_obj <- createCellChat(object = data.input, meta = meta, group.by = "group")
cc_obj <- addMeta(cc_obj, meta = meta, meta.name = "Labels")
cc_obj <- setIdent(cc_obj, ident.use = "Labels")
levels(cc_obj@idents)

CellChatDB <- CellChatDB.mouse
showDatabaseCategory(CellChatDB)
dplyr::glimpse(CellChatDB$interaction)
CellChatDB.use <- CellChatDB

cc_obj@DB <- CellChatDB.use
cc_obj <- subsetData(cc_obj)
cc_obj <- identifyOverExpressedGenes(cc_obj)
cc_obj <- identifyOverExpressedInteractions(cc_obj)

cc_obj <- projectData(cc_obj, PPI.mouse)
cc_obj <- computeCommunProb(cc_obj, raw.use = FALSE)

cc_obj <- filterCommunication(cc_obj, min.cells = 10)
cc_obj <- computeCommunProbPathway(cc_obj)
cc_obj <- aggregateNet(cc_obj)
cc_obj <- netAnalysis_computeCentrality(object = cc_obj, slot.name = "netP",  thresh = 0.5)

cc_LA_ALDO <- cc_obj

saveRDS(cc_obj, file = "CellChat/CellChat_LA_ALDO")

In [ ]:
groupSize <- as.numeric(table(cc_LA_ALDO@idents))

svg(file = "CellChat/Plots/LA_ALDO_Number.svg")
gg1 <- netVisual_circle(cc_LA_ALDO@net$count, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= FALSE, title.name = "Number of interactions", color.use = palette) 
dev.off()

svg(file = "CellChat/Plots/LA_ALDO_Weight.svg")
gg2 <- netVisual_circle(cc_LA_ALDO@net$weight, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= FALSE, title.name = "Interaction weights/strength", color.use = palette)
dev.off()

svg(file = "CellChat/Plots/LA_ALDO_byCluster.svg", width = 24, height = 18)
mat <- cc_LA_ALDO@net$weight
par(mfrow = c(3,5), xpd = TRUE)
for (i in 1:nrow(mat)) {
  mat1 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat1[i, ] <- mat[i, ]
  netVisual_circle(mat1, vertex.size.max = 5, vertex.weight = groupSize, vertex.label.cex = 3, arrow.size = 0, weight.scale = TRUE, edge.weight.max = max(mat), title.name = rownames(mat)[i], color.use = palette)
}
dev.off()

## CellChat LA CTRL

In [ ]:
data.input <- GetAssayData(split_LA$CTRL, assay = "RNA", slot = "data")
labels <- Idents(split_LA$CTRL)
meta <- data.frame(group = labels, row.names = names(labels))
cc_obj <- createCellChat(object = data.input, meta = meta, group.by = "group")
cc_obj <- addMeta(cc_obj, meta = meta, meta.name = "Labels")
cc_obj <- setIdent(cc_obj, ident.use = "Labels")
levels(cc_obj@idents)

CellChatDB <- CellChatDB.mouse
showDatabaseCategory(CellChatDB)
dplyr::glimpse(CellChatDB$interaction)
CellChatDB.use <- CellChatDB

cc_obj@DB <- CellChatDB.use
cc_obj <- subsetData(cc_obj)
cc_obj <- identifyOverExpressedGenes(cc_obj)
cc_obj <- identifyOverExpressedInteractions(cc_obj)

cc_obj <- projectData(cc_obj, PPI.mouse)
cc_obj <- computeCommunProb(cc_obj, raw.use = FALSE)

cc_obj <- filterCommunication(cc_obj, min.cells = 10)
cc_obj <- computeCommunProbPathway(cc_obj)
cc_obj <- aggregateNet(cc_obj)
cc_obj <- netAnalysis_computeCentrality(object = cc_obj, slot.name = "netP",  thresh = 0.5)


cc_LA_CTRL <- cc_obj

saveRDS(cc_obj, file = "CellChat/CellChat_LA_CTRL")

In [ ]:
groupSize <- as.numeric(table(cc_LA_CTRL@idents))

svg(file = "CellChat/Plots/LA_CTRL_Number.svg")
gg1 <- netVisual_circle(cc_LA_CTRL@net$count, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= FALSE, title.name = "Number of interactions", color.use = palette) 
dev.off()

svg(file = "CellChat/Plots/LA_CTRL_Weight.svg")
gg2 <- netVisual_circle(cc_LA_CTRL@net$weight, vertex.weight = groupSize, vertex.label.cex = 2, weight.scale = TRUE, label.edge= FALSE, title.name = "Interaction weights/strength", color.use = palette)
dev.off()

svg(file = "CellChat/Plots/LA_CTRL_byCluster.svg", width = 24, height = 18)
mat <- cc_LA_CTRL@net$weight
par(mfrow = c(3,5), xpd = TRUE)
for (i in 1:nrow(mat)) {
  mat1 <- matrix(0, nrow = nrow(mat), ncol = ncol(mat), dimnames = dimnames(mat))
  mat1[i, ] <- mat[i, ]
  netVisual_circle(mat1, vertex.size.max = 5, vertex.weight = groupSize, vertex.label.cex = 3, arrow.size = 0, weight.scale = TRUE, edge.weight.max = max(mat), title.name = rownames(mat)[i], color.use = palette)
}
dev.off()

## Part I: Predict general principles of cell-cell communication

In [ ]:
object.list <- list(LA_CTRL = cc_LA_CTRL, LA_ALDO = cc_LA_ALDO)

cellchat <- mergeCellChat(object.list, add.names = names(object.list))
# Merge the following slots: 'data.signaling','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.
cellchat

### Compare the total number of interactions and interaction strength

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)

gg1 <- compareInteractions(cellchat, show.legend = FALSE, group = c(1 ,2), color.use = palette.treatment, title.name = "Number of interactions", size.text = 20)
gg2 <- compareInteractions(cellchat, show.legend = FALSE, group = c(1 ,2), measure = "weight", color.use = palette.treatment, title.name = "Strength of interactions", size.text = 20)
gg1 + gg2

ggsave("CellChat/Plots/NumberOfInteractions.svg", plot = gg1, units = "cm", dpi = 300, width = 10, height = 10)
ggsave("CellChat/Plots/StrengthOfInteractions.svg", plot = gg2, units = "cm", dpi = 300, width = 10, height = 10)

### Differential number of interactions or interaction strength among different cell populations


In [ ]:
# LA
svg(file = "CellChat/Plots/LA_diffInteraction.svg", width = 14, height = 7)
par(mfrow = c(1, 2), xpd = TRUE)
PlotNetdiff_Num <- netVisual_diffInteraction(cellchat, weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, color.use = palette)
PlotNetdiff_Wght <- netVisual_diffInteraction(cellchat, weight.scale = TRUE, vertex.label.cex = 2, arrow.size = 0.5, measure = "weight", color.use = palette)
dev.off()

In [ ]:
# LA only form and to CM_0
target_cell <- "CM_0"


svg(file = "CellChat/Plots/LA_diffInteraction_CM_0_target.svg", width = 14, height = 7)
par(mfrow = c(1, 2), xpd = TRUE)
PlotNetdiff_Num <- netVisual_diffInteraction(cellchat, weight.scale = TRUE, targets.use = target_cell, vertex.label.cex = 2, arrow.size = 0.5, color.use = palette)
PlotNetdiff_Wght <- netVisual_diffInteraction(cellchat, weight.scale = TRUE, targets.use = target_cell, vertex.label.cex = 2, arrow.size = 0.5, measure = "weight", color.use = palette)
dev.off()

svg(file = "CellChat/Plots/LA_diffInteraction_CM_0_source.svg", width = 14, height = 7)
par(mfrow = c(1, 2), xpd = TRUE)
PlotNetdiff_Num <- netVisual_diffInteraction(cellchat, weight.scale = TRUE, sources.use = target_cell, vertex.label.cex = 2, arrow.size = 0.5, color.use = palette)
PlotNetdiff_Wght <- netVisual_diffInteraction(cellchat, weight.scale = TRUE, sources.use = target_cell, vertex.label.cex = 2, arrow.size = 0.5, measure = "weight", color.use = palette)
dev.off()

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6, repr.plot.res = 300)

ht1 <- netVisual_heatmap(cellchat, width = 10, height = 24, color.use = palette, color.heatmap = c("#0000c0", "#a00000"))
ht2 <- netVisual_heatmap(cellchat, width = 10, height = 24, measure = "weight", color.use = palette, color.heatmap = c("#0000c0", "#a00000"))

grob_ht1 = grid.grabExpr(draw(ht1))
grob_ht2 = grid.grabExpr(draw(ht2))

grid_ht <- plot_grid(grob_ht1, grob_ht2, ncol = 2)

ggsave("CellChat/Plots/netVisual_heatmap.svg", plot = grid_ht, units = "cm", dpi = 300, width = 30, height = 20)

In [ ]:
# Number of interactions combined in one plot

# LA
svg(file = "CellChat/Plots/LA_NumberOfInteractions.svg", width = 14, height = 7)
weight.max <- getMaxWeight(object.list, attribute = c("idents", "count"))
par(mfrow = c(1,2), xpd = TRUE)
for (i in 1:length(object.list)) {
  netVisual_circle(object.list[[i]]@net$count, weight.scale = TRUE, label.edge= FALSE, vertex.label.cex = 2, edge.weight.max = weight.max[2], edge.width.max = 12, title.name = paste0("Number of interactions - ", names(object.list)[i]), color.use = palette)
}
dev.off()

### Differential number of interactions or interaction strength among different cell types


### Compare the major sources and targets in 2D space

Comparing the outgoing and incoming interaction strength in 2D space allows ready identification of the cell populations with significant changes in sending or receiving signals between different datasets. 


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 300)

# lA
svg(file = "CellChat/Plots/LA_signalingRoleScatter.svg", width = 9, height = 4)
num.link <- sapply(object.list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
weight.MinMax <- c(min(num.link), max(num.link)) # control the dot size in the different datasets
gg <- list()
for (i in 1:length(object.list)) {
  gg[[i]] <- netAnalysis_signalingRole_scatter(object.list[[i]], title = names(object.list)[i], weight.MinMax = weight.MinMax, color.use = palette) + scale_y_continuous(limits = c(0, 15)) + scale_x_continuous(limits = c(0, 15))
}
patchwork::wrap_plots(plots = gg)
dev.off()

### Identify signaling changes associated with one cell group

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)

palette.net <- c(
    "#b57d7d", #share
    "#C1C1C1", #CTRL
    "#AF0000") #ALDO

In [ ]:
# Legend
gg0 <- netAnalysis_signalingChanges_scatter(cellchat, idents.use = "CM_0", color.use = palette.net, show.legend = TRUE)
ggsave("CellChat/Plots/scatterDiffPath/legend.svg", plot = gg0, units = "cm", dpi = 300, width = 15, height = 15)

In [ ]:
gg0

### Identify signaling changes associated with one cell group

In [ ]:
# LA
generate_and_save_plot <- function(ident, filename) {
  gg <- netAnalysis_signalingChanges_scatter(cellchat, idents.use = ident, color.use = palette.net, show.legend = FALSE)
  ggsave(filename, plot = gg, units = "cm", dpi = 300, width = 15, height = 15)
}

# List of identities and corresponding filenames
identities <- unique(LA@meta.data$cell_type)
filenames <- paste0("CellChat/Plots/scatterDiffPath/LA_", identities, ".svg")

# Generate and save scatter plots for each identity
Map(generate_and_save_plot, identities, filenames)

## Part II: Identify the conserved and context-specific signaling pathways

### Compare the overall information flow of each signaling pathway


In [ ]:
options(repr.plot.width = 7, repr.plot.height = 8, repr.plot.res = 300)

palette.rank <- c(
"#C1C1C1", #CTRL
"#AF0000") #ALDO

# LA
gg1 <- rankNet(cellchat, mode = "comparison", stacked = TRUE, do.stat = TRUE, color.use = palette.rank, sources.use = c("CM_0"), targets.use = c("FB"))
gg2 <- rankNet(cellchat, mode = "comparison", stacked = FALSE, do.stat = TRUE, color.use = palette.rank)
#ggsave("CellChat/Plots/LA_RelativeInformationFlow.svg", plot = gg1, units = "cm", dpi = 300, width = 15, height = 20)
#ggsave("CellChat/Plots/LA_InformationFlow.svg", plot = gg2, units = "cm", dpi = 300, width = 15, height = 20)

gg1 + gg2

### Compare outgoing (or incoming) signaling associated with each cell population
The above analysis summarize the information from the outgoing and incoming signaling together. We can also compare the outgoing (or incoming) signaling pattern between two datasets, allowing to identify signaling pathways/ligand-receptors that exhibit different signaling patterns.

We can combine all the identified signaling pathways from different datasets and thus compare them side by side, including outgoing signaling, incoming signaling and overall signaling by aggregating outgoing and incoming signaling together. **NB**: `rankNet` also shows the comparison of overall signaling, but it does not show the signaling strength in specific cell populations.


In [ ]:
# Heatmap showing the contribution of OUTGOING signals
i = 1

# LA
pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)
ht1 = netAnalysis_signalingRole_heatmap(object.list[[i]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i], width = 5, height = 25, color.use = palette, color.heatmap = "YlOrRd")
ht2 = netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "outgoing", signaling = pathway.union, title = names(object.list)[i+1], width = 5, height = 25, color.use = palette, color.heatmap = "YlOrRd")

In [ ]:
grob_ht1 = grid.grabExpr(draw(ht1))
grob_ht2 = grid.grabExpr(draw(ht2))

grid_ht_LA <- plot_grid(grob_ht1, grob_ht2, ncol = 2)

ggsave("CellChat/Plots/LA_netAnalysis_heatmap_OUT.svg", plot = grid_ht_LA, units = "cm", dpi = 300, width = 15, height = 30)

In [ ]:
# Heatmap showing the contribution of INCOMING signals
i = 1

# LA
pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)
ht1 = netAnalysis_signalingRole_heatmap(object.list[[i]], pattern = "incoming", signaling = pathway.union, title = names(object.list)[i], width = 5, height = 25, color.use = palette, color.heatmap = "YlOrRd")
ht2 = netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "incoming", signaling = pathway.union, title = names(object.list)[i+1], width = 5, height = 25, color.use = palette, color.heatmap = "YlOrRd")

In [ ]:
grob_ht1 = grid.grabExpr(draw(ht1))
grob_ht2 = grid.grabExpr(draw(ht2))

grid_ht_LA <- plot_grid(grob_ht1, grob_ht2, ncol = 2)

ggsave("CellChat/Plots/LA_netAnalysis_heatmap_INC.svg", plot = grid_ht_LA, units = "cm", dpi = 300, width = 15, height = 30)

In [ ]:
# Heatmap showing the contribution of ALL signals
i = 1

# LA
pathway.union <- union(object.list[[i]]@netP$pathways, object.list[[i+1]]@netP$pathways)
ht1 = netAnalysis_signalingRole_heatmap(object.list[[i]], pattern = "all", signaling = pathway.union, title = names(object.list)[i], width = 5, height = 25, color.use = palette, color.heatmap = "YlOrRd")
ht2 = netAnalysis_signalingRole_heatmap(object.list[[i+1]], pattern = "all", signaling = pathway.union, title = names(object.list)[i+1], width = 5, height = 25, color.use = palette, color.heatmap = "YlOrRd")

In [ ]:
grob_ht1 = grid.grabExpr(draw(ht1))
grob_ht2 = grid.grabExpr(draw(ht2))

grid_ht_LA <- plot_grid(grob_ht1, grob_ht2, ncol = 2)

ggsave("CellChat/Plots/LA_netAnalysis_heatmap_ALL.svg", plot = grid_ht_LA, units = "cm", dpi = 300, width = 15, height = 30)

## Part III: Identify the upgulated and down-regulated signaling ligand-receptor pairs


### Identify dysfunctional signaling by using differential expression analysis


In [ ]:
# LA
object.list <- list(LA_CTRL = cc_LA_CTRL, LA_ALDO = cc_LA_ALDO)
cellchat <- mergeCellChat(object.list, add.names = names(object.list))

In [ ]:
# LA
# define a positive dataset, i.e., the dataset with positive fold change against the other dataset
pos.dataset = "LA_ALDO"
# define a char name used for storing the results of differential expression analysis
features.name = paste0(pos.dataset, ".merged")

# perform differential expression analysis
cellchat <- identifyOverExpressedGenes(cellchat, group.dataset = "datasets", pos.dataset = pos.dataset, features.name = features.name, only.pos = FALSE, thresh.pc = 0.05, thresh.fc = 0.05, thresh.p = 0.1)
# map the results of differential expression analysis onto the inferred cell-cell communications to easily manage/subset the ligand-receptor pairs of interest
net_LA <- netMappingDEG(cellchat, features.name = features.name)

In [ ]:
# extract the ligand-receptor pairs with upregulated ligands in ALDO
net.up_LA <- subsetCommunication(cellchat, net = net_LA, datasets = "LA_ALDO", ligand.logFC = 0.05, receptor.logFC = NULL)
# extract the ligand-receptor pairs with upregulated ligands and upregulated recetptors in CTRL, i.e.,downregulated in ALDO
net.down_LA <- subsetCommunication(cellchat, net = net_LA, datasets = "LA_ALDO", ligand.logFC = -0.05, receptor.logFC = NULL)

In [ ]:
gene.up_LA <- extractGeneSubsetFromPair(net.up_LA, cellchat)
gene.down_LA <- extractGeneSubsetFromPair(net.down_LA, cellchat)

In [ ]:
# Get the unique cell groups from the metadata
unique(object.list[[2]]@meta$Labels)  # Or 'ident' depending on the column name

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5, repr.plot.res = 300)
#par(mfrow = c(1,2), xpd = TRUE)

chord1 <- netVisual_chord_gene(object.list[[1]], sources.use = c(1), targets.use = c(3, 9), slot.name = "net", net = net.down_LA, lab.cex = 0.8, small.gap = 1, annotationTrackHeight = c(0.05), title.name = paste0("Down-regulated signaling in ", names(object.list)[2]), color.use = palette)
chord2 <- netVisual_chord_gene(object.list[[2]], sources.use = c(1), targets.use = c(3, 9), slot.name = "net", net = net.up_LA, lab.cex = 0.8, small.gap = 1, annotationTrackHeight = c(0.05), title.name = paste0("Up-regulated signaling in ", names(object.list)[2]), color.use = palette)

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 3, repr.plot.res = 300)

# Display the plot
print(chord1)
g <- recordPlot()

svg("CellChat/Plots/LA_netVisual_chord_gene_plot_ALDO_Down.svg", width = 4, height = 4)
replayPlot(g)
dev.off()

In [ ]:
options(repr.plot.width = 3, repr.plot.height = 3, repr.plot.res = 300)

# Display the plot
print(chord2)
g <- recordPlot()

svg("CellChat/Plots/LA_netVisual_chord_gene_plot_ALDO_Up.svg", width = 4, height = 4)
replayPlot(g)
dev.off()

## netVisual_bubble CM, FB, TC 

In [ ]:
options(repr.plot.width = 4, repr.plot.height = 8, repr.plot.res = 300)

palette.bubble <- c( 
    "#000000", #CTRL
    "#AF0000" #ALDO   
)

gg1 <- netVisual_bubble(cellchat, sources.use = c(1, 9), targets.use = c(3, 9),  comparison = c(1, 2), max.dataset = 2, title.name = "Increased signaling in ALDO ", angle.x = 90, remove.isolate = TRUE, color.heatmap = "viridis", color.text = palette.bubble)


gg1 <- gg1 + scale_fill_gradientn(colors = brewer.pal(9, "OrRd")) +
      scale_color_gradientn(colors = brewer.pal(9, "OrRd"))

ggsave("CellChat/Plots/LA_netVisual_bubble_CM_FB_TC.svg", plot = gg1, units = "cm", dpi = 300, width = 9.8, height = 14)

gg1

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8, repr.plot.res = 300)

netVisual_bubble(cellchat, sources.use = c(1, 9), targets.use = c(3, 9),  comparison = c(1, 2), max.dataset = 2, title.name = "Increased signaling in ALDO ", angle.x = 90, remove.isolate = TRUE, color.heatmap = "viridis", color.text = palette.bubble)|
netVisual_bubble(cellchat, sources.use = c(1, 9), targets.use = c(3, 9),  comparison = c(2, 1), max.dataset = 2, title.name = "Increased signaling in ALDO ", angle.x = 90, remove.isolate = TRUE, color.heatmap = "viridis", color.text = palette.bubble)

In [2]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 24.04.1 LTS

Matrix products: default
BLAS/LAPACK: /media/daten/dmeral/micromamba/envs/user_R/lib/libopenblasp-r0.3.26.so

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

attached base packages:
[1] grid      stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] RColorBrewer_1.1-3          pals_1.8                   
 [3] gridExtra_2.3               cowplot_1.1.3              
 [5] reticulate_1.35.0           patchwork_1.3.0            
 [7] CellChat_1.6.1              igraph_2.1.4               
 [9] circlize_0.4.16             ComplexHeatmap_2.14.0      
[11] viridis_0.6.5        